In [165]:
%%writefile plag_finder.py

from ipykernel.kernelbase import Kernel
import os
from io import BytesIO
import urllib, base64

Overwriting plag_finder.py


In [166]:
%%writefile plag_finder.py -a
import math
import string
import sys

# reading the text file
# This functio will return a
# list of the lines of text
# in the file.
def read_file(filename):

    try:
        with open(filename, 'r') as f:
            data = f.read()
        return data

    except IOError:
        print("Error opening or reading input file: ", filename)
        sys.exit()

# splitting the text lines into words
# translation table is a global variable
# mapping upper case to lower case and
# punctuation to spaces
translation_table = str.maketrans(string.punctuation+string.ascii_uppercase,
                                    " "*len(string.punctuation)+string.ascii_lowercase)

# returns a list of the words
# in the file
def get_words_from_line_list(text):

    text = text.translate(translation_table)
    word_list = text.split()

    return word_list


# counts frequency of each word
# returns a dictionary which maps
# the words to their frequency.
def count_frequency(word_list):

    D = {}

    for new_word in word_list:

        if new_word in D:
            D[new_word] = D[new_word] + 1

        else:
            D[new_word] = 1

    return D

# returns dictionary of (word, frequency)
# pairs from the previous dictionary.
def word_frequencies_for_file(filename):

    line_list = read_file(filename)
    word_list = get_words_from_line_list(line_list)
    freq_mapping=0
    freq_mapping = count_frequency(word_list)

    return freq_mapping


# returns the dot product of two documents
def dotProduct(D1, D2):
    Sum = 0.0

    for key in D1:

        if key in D2:
            Sum += (D1[key] * D2[key])

    return Sum

# returns the angle in radians
# between document vectors
def vector_angle(D1, D2):
    numerator = dotProduct(D1, D2)
    denominator = math.sqrt(dotProduct(D1, D1)*dotProduct(D2, D2))

    return math.acos(numerator / denominator)


def documentSimilarity(filename_1, filename_2):

    # filename_1 = sys.argv[1]
    # filename_2 = sys.argv[2]
    sorted_word_list_1 = word_frequencies_for_file(filename_1)
    sorted_word_list_2 = word_frequencies_for_file(filename_2)
    distance=0
    distance = vector_angle(sorted_word_list_1, sorted_word_list_2)
    return distance





Appending to plag_finder.py


In [167]:
%%writefile plag_finder.py -a

class plag_finder(Kernel):
    implementation = 'Plot'
    implementation_version = '1.0'
    language = 'python'  # will be used for
                         # syntax highlighting
    language_version = '3.6'
    language_info = {'name': 'plotter',
                     'mimetype': 'text/plain',
                     'extension': '.py'}
    banner = "Simple plotting"

Appending to plag_finder.py


In [168]:
%%writefile plag_finder.py -a

    def do_execute(self, code, silent,
                   store_history=True,
                   user_expressions=None,
                   allow_stdin=False):

        
        
        path =code.split('\n')[0]
        files = os.listdir(path)
        result=""
        n_files=len(files)
        for i in range(1,n_files-1):
            for j in range(i+1,n_files):
                file_1=path+"/"+files[i]
                file_2=path+"/"+files[j]
                ans=documentSimilarity(file_1,file_2)
                deg = math.degrees(ans);
                perc=100-((deg*100)/90);
                result+="Plagarism between file :-" + file_1 +" and file :-" + file_2 +" is = " + str(perc) +"%\n"
                

        if not silent:
            # We send the standard output to the
            # client.
            self.send_response(
                self.iopub_socket,
                'stream', {
                    'name': 'stdout',
                    'data': ('Plotting {n} ')})

            # We prepare the response with our rich
            # data (the plot).
            content = {
                'source': 'kernel',

                # This dictionary may contain
                # different MIME representations of
                # the output.
                'data': {
                        'text/plain' : result
                },

                
            }

            # We send the display_data message with
            # the contents.
            self.send_response(self.iopub_socket,
                'display_data', content)

        # We return the exection results.
        return {'status': 'ok',
                'execution_count':
                    self.execution_count,
                'payload': [],
                'user_expressions': {},
               }

Appending to plag_finder.py


In [169]:
%%writefile plag_finder.py -a

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(
        kernel_class=plag_finder)

Appending to plag_finder.py


In [170]:
%mkdir -p plag_files/

In [171]:

%%writefile plag_files/kernel.json
{
 "argv": ["python", "-m",
          "plag_finder", "-f",
          "{connection_file}"],
 "display_name": "plag_finder",
 "name": "plag_finder",
 "language": "python"
}

Overwriting plag_files/kernel.json


In [172]:
!jupyter kernelspec install --user plag_files

[InstallKernelSpec] Removing existing kernelspec in /Users/anshulkulhari/Library/Jupyter/kernels/plag_files
[InstallKernelSpec] Installed kernelspec plag_files in /Users/anshulkulhari/Library/Jupyter/kernels/plag_files


In [173]:
!jupyter kernelspec list

Available kernels:
  python3          /usr/local/Cellar/jupyterlab/3.0.14/libexec/lib/python3.9/site-packages/ipykernel/resources
  client_kernel    /Users/anshulkulhari/Library/Jupyter/kernels/client_kernel
  githubkernel     /Users/anshulkulhari/Library/Jupyter/kernels/githubkernel
  grammar          /Users/anshulkulhari/Library/Jupyter/kernels/grammar
  plag_files       /Users/anshulkulhari/Library/Jupyter/kernels/plag_files
  plotter          /Users/anshulkulhari/Library/Jupyter/kernels/plotter
  server_kernel    /Users/anshulkulhari/Library/Jupyter/kernels/server_kernel
